# Load Depenency

In [109]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [110]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [111]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [112]:
disease=input()
print(disease)

cancer


In [113]:
# disease = 'parkinson'

#disease = 'depression'

# disease = 'diabetes'

# disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [114]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [115]:
type(infodf)

dict

In [116]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

101358


In [117]:
count = ct.get_study_count(search_expr=disease)


In [118]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )





temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


In [119]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"]

In [120]:
# new_df
new_df.shape

(101358, 20)

In [121]:
new_df.head

<bound method NDFrame.head of       Index        NCTId                                          Condition  \
0         1  NCT04318756                      Cancer Fear|Pancreatic Cancer   
1         2  NCT03581708              Lung Neoplasms|Venous Thromboembolism   
2         3  NCT04834570                                      Breast Cancer   
3         4  NCT00897650                                        Lung Cancer   
4         5  NCT04494620                                        Oral Cancer   
..      ...          ...                                                ...   
454  101354  NCT03607786  Type A Aortic Dissection|Circulatory Arrest|Hy...   
455  101355  NCT02686996                                Insulin Sensitivity   
456  101356  NCT04391309  Coronavirus Disease 2019 (COVID-19)|Severe Acu...   
457  101357  NCT05212714                   Stress Disorders, Post-Traumatic   
458  101358  NCT04183179  Healthy Diet|Stress|Pediatric Obesity|Intergen...   

                     

In [122]:
# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

In [123]:
# print(temp.shape)

In [124]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


In [125]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [126]:
new_df2.shape

(101358, 21)

In [127]:
new_df2

,index,NCTId,IsFDARegulatedDrug,IsFDARegulatedDevice,IsUnapprovedDevice,PrimaryOutcomeMeasure,PrimaryOutcomeDescription,PrimaryOutcomeTimeFrame,SecondaryOutcomeMeasure,SecondaryOutcomeDescription,...,OtherOutcomeMeasure,OtherOutcomeDescription,OtherOutcomeTimeFrame,EligibilityCriteria,StudyPopulation,HealthyVolunteers,ReferencePMID,LocationCity,LocationState,LocationFacility
0,1,NCT04318756,No,No,,Syntactic and semantic equivalence of Cancer w...,the Syntactic and semantic equivalence of scal...,1 month,Face Validity of cancer worry scale on Italian...,Face Validity will be measured through the eva...,...,,,,Inclusion Criteria:||Subject will be recruited...,Subject will be recruited from the current out...,No,,,,
1,2,NCT03581708,No,No,,VTE incidence risk evaluation,Real world VTE incidence risk evaluation in ad...,12 months,VTE incidence|VTE and overall survival,VTE incidence in advanced lung cancer.|The inf...,...,,,,Inclusion Criteria:||Age ≥ 18 years at the tim...,Patients diagnosed with advanced staged lung c...,No,18216292|15701913|17940477|10499070|12083490|1...,Guangzhou,Guagndong,Guangdong General Hospital
2,3,NCT04834570,No,No,,Change in Global Health Status by EORTC QLQ-C3...,This questionnaire was designed to be cancer s...,1 year after treatment ends; 2 years after tre...,Quality of life specific for breast cancer by ...,"The questionnaire contains 23-item, two breast...",...,,,,Inclusion Criteria:||Breast cancer|Adult women...,Recruitment of 100 breast cancer patients who ...,No,,São Paulo,,IBCC Oncologia
3,4,NCT00897650,,,,"DNA, RNA, and Protein expression patterns and ...","DNA, RNA, and Protein expression patterns and ...",After lung tumor tissue and blood collection.,Characterization of genes and proteins predict...,Identify genes and proteins predictive of resp...,...,,,,Inclusion criteria||Diagnosis of suspected lun...,People who have or may have lung cancer.,No,,Nashville,Tennessee,Vanderbilt-Ingram Cancer Center
4,5,NCT04494620,No,No,,Oral Cancer Death Rate,Information on all deaths were collected from ...,15 years,Oral Cavity Cancer Incidence,Information on incident oral cancer cases were...,...,,,,Inclusion Criteria: Residing in 13 study clust...,,Accepts Healthy Volunteers,23265945|15936419|33449824,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,101354,NCT03607786,No,No,,Composite major complications,Newly Postoperative Renal Failure;Surgical Re-...,up to 12 months,Respiratory Failure|Transient Neurological Def...,"The occurrence of postoperative agitation, con...",...,The number of volume of blood products transfu...,The number of units of packed red blood cells ...,up to 30 days|up to 30 days|up to 30 days|up t...,Inclusion Criteria:||Patient aged 18 yr-65yr|P...,,No,33879045|31014386,Chengdu,Sichuan,West China Hospital of Sichuan University
455,101355,NCT02686996,No,No,,Change in insulin sensitivity measured by eugl...,The clamp will be used to measure insulin sens...,From baseline to 14 weeks,Change in markers of endothelial dysfunction|C...,This is done using non-invasive peripheral art...,...,Change in Arterial waveform measurement|Change...,"This is done with the BP+ device (Uscom Ltd, A...",From baseline to 14 weeks|From baseline to 14 ...,"Inclusion Criteria:||Age >18 or <60 years,|Wei...",,Accepts Healthy Volunteers,33986049,Melbourne,Victoria,Monash Centre for Health Research and Implemen...
456,101356,NCT04391309,Yes,No,,Time to Clinical Recovery,The Primary Endpoint is time to clinical recov...,Day 0-28,Days Alive and Free of Any Episodes of Acute R...,Defined by need for any of the following oxyge...,...,EXPLORATORY: Change in Sequential Organ Failur...,Sequential Organ Failure Assessment score (ran...,"Days 0-7, Days 0-14, Days 0-21, and Days 0-28|...",Inclusion Criteria:||Patients included in the ...,,No,,Torrance|Miami Beach|Sarasota|Atlanta|New Orle...,California|Florida|Florida|Georgia|Louisiana|M...,Harbor - UCLA Medical Center|Universi

In [128]:




# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId"],
#     max_studies=10,
#     min_rnk=5,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])
# temp

In [129]:




# # Get the NCTId, Condition and Brief title fields from 500 studies related to Coronavirus and Covid, in csv format.
# api_pull_1 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
#     "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" 
#     ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","IsFDARegulatedDrug","IsFDARegulatedDevice","BriefSummary"],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# df1=pd.DataFrame.from_records(api_pull_1[1:], columns=api_pull_1[0])

# # 
# # df1 = pd.DataFrame.to_frame().reset_index()



In [130]:
# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
# ,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"

# ],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records

# ct.get_study_count(search_expr="Parkinson")

# # Read the csv data in Pandas
# import pandas as pd

# df2 = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

##cleaning

In [131]:
# print(type(df1))

In [132]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='outer')

In [133]:
result.shape

(101358, 40)

In [134]:
# result = result.loc[:,~result.columns.duplicated()]

In [135]:
# result.shape

In [136]:
# result.columns.tolist()

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [137]:
result = result.drop(result.columns[[20]], axis=1)

In [138]:
result.shape

(101358, 39)

In [139]:
# result.columns.tolist()

Save file

In [140]:
# result.to_csv(r'data_raw.csv', index = False)
result.to_excel(r'data_raw.xlsx', index = False)

In [141]:
result.to_hdf('data_raw.h5', key='df', mode='w')  

In [142]:
# labels, levels = pd.factorize(result["Phase"])  

In [143]:
# labels

In [144]:
# fun_barplot()